### About

This notebooks imports a CSV dataset into Azure SQL Server table.


In [160]:
%%pyspark
df = spark.read.load('abfss://filesystem@<<Azure Synapse Workspace>>.dfs.core.windows.net/HotelReviews.csv', format='csv'
, sep = '|'
, header=True
)
display(df.limit(10))

StatementMeta(sparkpool1, 7, 5, Finished, Available)

SynapseWidget(Synapse.DataFrame, f187bdff-6e89-499b-8225-43a1ee722bbe)

### Note

Spark MS SQL Connector with spark 3.3

<b>Error</b>: com.microsoft.sqlserver.jdbc.spark

<b>Resolution</b>: 
Download on computer from here: https://repo1.maven.org/maven2/com/microsoft/azure/spark-mssql-connector_2.12/1.3.0-BETA/spark-mssql-connector_2.12-1.3.0-BETA.jar

Then upload to workspace (Manage->Worksspace packages->Upload)

Then add package to spark pool(Manage->Apache spark pool->Select the pool name->click...->Packages and select the uploaded workspace package)


### SQL Connection Configuration

Ref: https://learn.microsoft.com/en-us/azure/synapse-analytics/spark/data-sources/apache-spark-sql-connector

Ref: https://learn.microsoft.com/en-us/sql/connect/spark/connector?view=sql-server-ver16

#### AKV
Added as a linked service to the workspace (Granted workspace MI access to AKV)

In [161]:
# The servername is in the format "jdbc:sqlserver://<AzureSQLServerName>.database.windows.net:1433"
servername = "jdbc:sqlserver://<<Azure SQL Server>>.database.windows.net:1433"
dbname = "<<Azure SQL DB>>"
url = servername + ";" + "databaseName=" + dbname + ";"
dbtable = "(SELECT TOP 5 * FROM Comments_coded) AS Comments_coded" #Update/remove TOP
dbtable_out = "HotelReviews" #Output table
user = mssparkutils.credentials.getSecret('<<Azure Key Vault>>','AZSQLUSR')
password = mssparkutils.credentials.getSecret('<<Azure Key Vault>>','AZSQLPWD')


StatementMeta(sparkpool1, 7, 6, Finished, Available)

### Load into Azure SQL table

In [162]:
try:
  df.write \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .mode("overwrite") \
    .option("url", url) \
    .option("dbtable", dbtable_out) \
    .option("user", user) \
    .option("password", password) \
    .save()
except ValueError as error :
    print("MSSQL Connector write failed", error)

print("MSSQL Connector write(overwrite) succeeded  ")

StatementMeta(sparkpool1, 7, 7, Finished, Available)

MSSQL Connector write(overwrite) succeeded  
